In [1]:
import sqlite3
import pandas as pd

import os, sys
import httpcore, httpx, requests, urllib3
import logging, time


main_dir = os.path.abspath('..')
sys.path.append(main_dir)
# os.chdir(main_dir)

# logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s:%(name)s:%(filename)s - %(message)s',
    handlers=[logging.FileHandler(
        f'{main_dir}/logs/log_{time.strftime("%Y%m%d")}.log'), logging.StreamHandler()]
)

pkgs_warn_only = ['urllib3', 'requests', 'httpx', 'httpcore']
[logging.getLogger(pkg).setLevel(logging.WARNING) for pkg in pkgs_warn_only]

notebook_name = __vsc_ipynb_file__[__vsc_ipynb_file__.rfind('/')+1:]
log = logging.getLogger(f'{notebook_name}_logger')
log.setLevel(logging.DEBUG)

In [8]:
from importlib import reload
import utils.llms
reload(utils.llms)

from utils.llms import CustomOpenAIClient

llm = CustomOpenAIClient(enable_streaming=False)

messages = [
    {"role": "system", "content": "You are a travel agent. Be descriptive and helpful."},
    {"role": "user", "content": "Tell me the top 3 things to do in Colorado."},
]


llm.generate(messages=messages)

2025-05-09 22:45:46,804 - DEBUG:openai._base_client:_base_client.py - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a travel agent. Be descriptive and helpful.'}, {'role': 'user', 'content': 'Tell me the top 3 things to do in Colorado.'}], 'model': 'meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8', 'max_tokens': 512, 'stream': False, 'temperature': 0.8, 'tools': None, 'top_p': 0.9}}
2025-05-09 22:45:46,814 - DEBUG:openai._base_client:_base_client.py - Sending HTTP Request: POST https://api.together.xyz/v1/chat/completions
2025-05-09 22:45:52,465 - DEBUG:openai._base_client:_base_client.py - HTTP Response: POST https://api.together.xyz/v1/chat/completions "200 OK" Headers({'date': 'Sat, 10 May 2025 04:45:52 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'cf-ray': '93d6ca002ee3e755-DEN', 'cf-cache-status': 'DYNAMIC', 'r

"Colorado! The Centennial State is a paradise for outdoor enthusiasts, foodies, and adventure-seekers alike. With its breathtaking Rocky Mountains, charming towns, and vibrant cities, Colorado has something for everyone. Here are the top 3 things to do in Colorado:\n\n1. **Explore the Rocky Mountain National Park**: A must-visit destination in Colorado, Rocky Mountain National Park is a stunning showcase of alpine grandeur. With over 60 mountain peaks above 12,000 feet, including Longs Peak, the park offers endless opportunities for hiking, camping, and wildlife viewing. Take a scenic drive on the Trail Ridge Road, the highest paved road in the US, and marvel at the breathtaking vistas. Keep an eye out for elk, moose, and bighorn sheep roaming freely in their natural habitat.\n\nSome popular trails in the park include:\n\t* Bear Lake Trail: A moderate 3.5-mile loop around Bear Lake, offering stunning views of the surrounding mountains.\n\t* Alberta Falls Trail: A 3.8-mile out-and-back 

In [7]:
import json
from utils.llms import CustomOpenAIClient
from utils.dummy_apis import get_referal_code
from utils.agentic_tools import GET_REFERRAL_CODE_TOOL


llm = CustomOpenAIClient(enable_streaming=True)

messages = [
    {"role": "system", "content": "You are a helpful assistant that can access external functions. The responses from these function calls will be appended to this dialogue. Please provide responses based on the information from these function calls.\n\nIf you don't know the user ID, ask the user to provide it."},
    {"role": "user", "content": "Can I get a referal code? My user ID is 3431235UK."},
]
    
# Completion #1: Get the appropriate tool calls
response = llm.generate(
    messages=messages,
    tools=GET_REFERRAL_CODE_TOOL,
)

tool_calls = response.choices[0].message.tool_calls
if tool_calls:
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "get_referal_code":
            function_response = get_referal_code(
                user_id=function_args.get("user_id"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

# Completion #2: Provide the results to get the final answer
function_enriched_response = llm.generate(
    messages=messages,
)
    
function_enriched_response


2025-05-09 23:48:17,539 - DEBUG:openai._base_client:_base_client.py - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are a helpful assistant that can access external functions. The responses from these function calls will be appended to this dialogue. Please provide responses based on the information from these function calls.\n\nIf you don't know the user ID, ask the user to provide it."}, {'role': 'user', 'content': 'Can I get a referal code? My user ID is 3431235UK.'}], 'model': 'meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8', 'max_tokens': 512, 'stream': False, 'temperature': 0.8, 'tools': [{'type': 'function', 'function': {'name': 'get_referal_code', 'description': 'Get the referal code for a user', 'parameters': {'type': 'object', 'properties': {'user_id': {'type': 'string', 'description': 'The user ID for which to get the referal code.'}}}}}], 'top_p': 0.9}}
2025-05-09 23:48:17,540 

2025-05-09 23:48:18,498 - DEBUG:openai._base_client:_base_client.py - HTTP Response: POST https://api.together.xyz/v1/chat/completions "200 OK" Headers({'date': 'Sat, 10 May 2025 05:48:18 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'cf-ray': '93d725921e2de673-DEN', 'cf-cache-status': 'DYNAMIC', 'retry-after': '2', 'access-control-allow-origin': '*', 'etag': 'W/"319-FdCPPChAeAEplwzP0y1i3ztPE3k"', 'strict-transport-security': 'max-age=15552000; includeSubDomains', 'x-amzn-trace-id': '338c0936-df59-41f3-90dc-f6e83147861c-noamzn', 'x-api-call-end': '2025-05-10T05:48:18.483Z', 'x-api-call-start': '2025-05-10T05:48:17.777Z', 'x-api-received': '2025-05-10T05:48:17.768Z', 'x-inference-version': 'v2', 'x-ratelimit': 'false', 'x-ratelimit-limit': '10', 'x-ratelimit-limit-tokens': '3000', 'x-ratelimit-remaining': '19', 'x-ratelimit-remaining-tokens': '3000', 'x-ratelimit-reset': '2', 'x-request-id': 'ntSh7KD-zqrih-93d725921e

In [8]:
[_ for _ in function_enriched_response]

[ChatCompletionChunk(id='ntSh7Z5-zqrih-93d72597d9d2e673', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None, token_id=None), finish_reason=None, index=0, logprobs=None, text='')], created=1746856098, model='meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None),
 ChatCompletionChunk(id='ntSh7Z5-zqrih-93d72597d9d2e673', choices=[Choice(delta=ChoiceDelta(content='Your', function_call=None, role='assistant', tool_calls=None, token_id=None), finish_reason=None, index=0, logprobs=None, text='Your')], created=1746856098, model='meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None),
 ChatCompletionChunk(id='ntSh7Z5-zqrih-93d72597d9d2e673', choices=[Choice(delta=ChoiceDelta(content=' referral', function_call=None, role='assistant', tool_calls=None, token_id=None), finish_reaso

In [9]:
"".join([chunk.choices[0].delta.content for chunk in function_enriched_response if chunk.choices])

''